In [ ]:

!pip install librosa noisereduce soundfile

import librosa
import librosa.display
import noisereduce as nr
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
from google.colab import files
import os
import zipfile
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# НАСТРОЙКИ ШУМОПОДАВЛЕНИЯ
settings = {
    'noise_reduction': True,
    'noise_duration': 0.5,
    'stationary': False,
    'prop_decrease': 1.0,
    'use_auto_noise': True,
    'output_format': 'wav',
}

print("Настройки шумоподавления:")
for key, value in settings.items():
    print(f"  {key}: {value}")

advanced_settings = {
    'time_constant_s': 2.0,
    'freq_mask_smooth_hz': 500,
    'time_mask_smooth_ms': 50,
    'n_fft': 2048,
    'hop_length': 512,
}

Настройки шумоподавления:
  noise_reduction: True
  noise_duration: 0.5
  stationary: False
  prop_decrease: 1.0
  use_auto_noise: True
  output_format: wav


In [ ]:

def reduce_noise_advanced(audio, sr, settings):
    """Продвинутое шумоподавление"""
    try:

        if settings['use_auto_noise']:

            noise_samples = int(settings['noise_duration'] * sr)
            noise_sample = audio[:min(noise_samples, len(audio)//10)]
        else:
            noise_sample = None


        if noise_sample is not None and len(noise_sample) > 100:
            cleaned_audio = nr.reduce_noise(
                y=audio,
                y_noise=noise_sample,
                sr=sr,
                stationary=settings['stationary'],
                prop_decrease=settings['prop_decrease'],
                time_constant_s=advanced_settings['time_constant_s'],
                freq_mask_smooth_hz=advanced_settings['freq_mask_smooth_hz'],
                time_mask_smooth_ms=advanced_settings['time_mask_smooth_ms'],
                n_fft=advanced_settings['n_fft'],
                hop_length=advanced_settings['hop_length']
            )
        else:

            cleaned_audio = nr.reduce_noise(
                y=audio,
                sr=sr,
                stationary=settings['stationary'],
                prop_decrease=settings['prop_decrease']
            )

        return cleaned_audio

    except Exception as e:
        print(f"Ошибка при шумоподавлении: {e}")
        return audio

def process_audio_file(input_path, output_path, settings):
    """Обработка одного аудиофайла"""
    try:

        audio, sr = librosa.load(input_path, sr=None)
        original_duration = len(audio) / sr

        if settings['noise_reduction']:
            processed_audio = reduce_noise_advanced(audio, sr, settings)
        else:
            processed_audio = audio

        sf.write(output_path, processed_audio, sr)

        processed_audio_check, _ = librosa.load(output_path, sr=None)
        processed_duration = len(processed_audio_check) / sr

        return {
            'success': True,
            'original_duration': original_duration,
            'processed_duration': processed_duration,
            'file_size_original': os.path.getsize(input_path),
            'file_size_processed': os.path.getsize(output_path)
        }

    except Exception as e:
        print(f"Ошибка при обработке {input_path}: {e}")
        return {'success': False, 'error': str(e)}

def plot_audio_analysis_fixed(original, processed, sr, title):
    """Исправленный анализ аудио до и после обработки"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 10))

    times = np.arange(len(original)) / sr
    ax1.plot(times, original, alpha=0.7, color='blue', label='Оригинал')
    ax1.plot(times, processed, alpha=0.7, color='red', label='Обработанный')
    ax1.set_title(f'Волновая форма - {title}')
    ax1.set_ylabel('Амплитуда')
    ax1.legend()
    ax1.grid(True)


    S_orig = librosa.stft(original)
    S_db_orig = librosa.amplitude_to_db(np.abs(S_orig), ref=np.max)
    img1 = librosa.display.specshow(S_db_orig, sr=sr, x_axis='time', y_axis='hz', ax=ax2)
    ax2.set_title('Спектрограмма оригинала')
    plt.colorbar(img1, ax=ax2, format='%+2.0f dB')

    S_proc = librosa.stft(processed)
    S_db_proc = librosa.amplitude_to_db(np.abs(S_proc), ref=np.max)
    img2 = librosa.display.specshow(S_db_proc, sr=sr, x_axis='time', y_axis='hz', ax=ax3)
    ax3.set_title('Спектрограмма после обработки')
    plt.colorbar(img2, ax=ax3, format='%+2.0f dB')

    S_diff = S_db_orig - S_db_proc
    img3 = librosa.display.specshow(S_diff, sr=sr, x_axis='time', y_axis='hz', ax=ax4)
    ax4.set_title('Удаленный шум')
    plt.colorbar(img3, ax=ax4, format='%+2.0f dB')

    plt.tight_layout()
    plt.show()

In [ ]:
print("Загрузите аудиофайлы для обработки (можно выбрать несколько)")
print("Поддерживаемые форматы: wav, mp3, flac, m4a, ogg")
uploaded = files.upload()

if not uploaded:
    print(" Файлы не загружены!")
else:
    print(f"\n Загружено файлов: {len(uploaded)}")
    file_info = []
    for filename in uploaded.keys():
        file_size = len(uploaded[filename]) / 1024 / 1024
        file_info.append((filename, file_size))
        print(f"  - {filename} ({file_size:.2f} MB)")

    for filename in uploaded.keys():
        with open(filename, 'wb') as f:
            f.write(uploaded[filename])

In [ ]:
if uploaded:
    print("\n Начинаем пакетную обработку...")

    processed_files = []
    processing_stats = []

    output_folder = "processed_audio"
    os.makedirs(output_folder, exist_ok=True)

    for filename in tqdm(uploaded.keys(), desc="Обработка файлов"):
        try:
            input_path = filename
            output_filename = f"cleaned_{os.path.splitext(filename)[0]}.{settings['output_format']}"
            output_path = os.path.join(output_folder, output_filename)

            result = process_audio_file(input_path, output_path, settings)

            if result['success']:
                processed_files.append(output_path)
                processing_stats.append({
                    'filename': filename,
                    'original_duration': result['original_duration'],
                    'processed_duration': result['processed_duration'],
                    'file_size_original': result['file_size_original'],
                    'file_size_processed': result['file_size_processed']
                })

                print(f" {filename} -> {output_filename}")
            else:
                print(f" Ошибка при обработке {filename}")

        except Exception as e:
            print(f" Критическая ошибка при обработке {filename}: {e}")

    print(f"\n Обработка завершена: {len(processed_files)}/{len(uploaded)} файлов")

In [ ]:
if processed_files:
    print("\n Предпросмотр результатов для первого файла:")

    first_original = list(uploaded.keys())[0]
    first_processed = processed_files[0]

    original_audio, sr = librosa.load(first_original, sr=None)
    processed_audio, _ = librosa.load(first_processed, sr=None)

    def plot_audio_analysis_fixed(original, processed, sr, title):
        """Исправленный анализ аудио до и после обработки"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 10))

        times = np.arange(len(original)) / sr
        ax1.plot(times, original, alpha=0.7, color='blue', label='Оригинал')
        ax1.plot(times, processed, alpha=0.7, color='red', label='Обработанный')
        ax1.set_title(f'Волновая форма - {title}')
        ax1.set_ylabel('Амплитуда')
        ax1.legend()
        ax1.grid(True)

        S_orig = librosa.stft(original)
        S_db_orig = librosa.amplitude_to_db(np.abs(S_orig), ref=np.max)
        img1 = librosa.display.specshow(S_db_orig, sr=sr, x_axis='time', y_axis='hz', ax=ax2)
        ax2.set_title('Спектрограмма оригинала')
        plt.colorbar(img1, ax=ax2, format='%+2.0f dB')

        S_proc = librosa.stft(processed)
        S_db_proc = librosa.amplitude_to_db(np.abs(S_proc), ref=np.max)
        img2 = librosa.display.specshow(S_db_proc, sr=sr, x_axis='time', y_axis='hz', ax=ax3)
        ax3.set_title('Спектрограмма после обработки')
        plt.colorbar(img2, ax=ax3, format='%+2.0f dB')

        S_diff = S_db_orig - S_db_proc
        img3 = librosa.display.specshow(S_diff, sr=sr, x_axis='time', y_axis='hz', ax=ax4)
        ax4.set_title('Удаленный шум')
        plt.colorbar(img3, ax=ax4, format='%+2.0f dB')

        plt.tight_layout()
        plt.show()

    plot_audio_analysis_fixed(original_audio, processed_audio, sr, first_original)

    print("Оригинальное аудио:")
    display(Audio(original_audio, rate=sr))

    print("Обработанное аудио:")
    display(Audio(processed_audio, rate=sr))

In [ ]:
if processed_files:
    print("\n Создание архива с обработанными файлами...")

    zip_filename = "noise_reduced_audio_files.zip"

    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file_path in processed_files:
            zipf.write(file_path, os.path.basename(file_path))

    zip_size = os.path.getsize(zip_filename) / 1024 / 1024
    print(f" Создан архив: {zip_filename} ({zip_size:.2f} MB)")

    print("\n Содержимое архива:")
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
        for file_info in zipf.filelist:
            print(f"  - {file_info.filename}")

    print("\n⬇  Скачивание архива...")
    files.download(zip_filename)

In [ ]:
if processing_stats:
    print("\n ДЕТАЛЬНАЯ СТАТИСТИКА ОБРАБОТКИ")
    print("=" * 60)

    total_original_duration = 0
    total_processed_duration = 0
    total_original_size = 0
    total_processed_size = 0

    for stat in processing_stats:
        original_dur = stat['original_duration']
        processed_dur = stat['processed_duration']
        original_size = stat['file_size_original'] / 1024 / 1024
        processed_size = stat['file_size_processed'] / 1024 / 1024

        total_original_duration += original_dur
        total_processed_duration += processed_dur
        total_original_size += original_size
        total_processed_size += processed_size

        duration_diff = original_dur - processed_dur
        duration_reduction = (duration_diff / original_dur * 100) if original_dur > 0 else 0

        print(f" {stat['filename']}:")
        print(f"     Длительность: {original_dur:.1f}с → {processed_dur:.1f}с")
        print(f"    Размер: {original_size:.2f}MB → {processed_size:.2f}MB")
        print(f"    Сокращение: {duration_reduction:.1f}%")
        print()

    total_duration_reduction = ((total_original_duration - total_processed_duration) /
                               total_original_duration * 100) if total_original_duration > 0 else 0
    total_size_reduction = ((total_original_size - total_processed_size) /
                           total_original_size * 100) if total_original_size > 0 else 0

    print(" ОБЩИЕ РЕЗУЛЬТАТЫ:")
    print(f"    Файлов обработано: {len(processing_stats)}")
    print(f"    Общая длительность: {total_original_duration:.1f}с → {total_processed_duration:.1f}с")
    print(f"    Общий размер: {total_original_size:.2f}MB → {total_processed_size:.2f}MB")
    print(f"    Общее сокращение: {total_duration_reduction:.1f}%")
    print(f"    Экономия места: {total_original_size - total_processed_size:.2f}MB")

In [ ]:

print("\n Очистка временных файлов...")

cleaned_files = 0
for filename in uploaded.keys():
    if os.path.exists(filename):
        os.remove(filename)
        cleaned_files += 1

if os.path.exists("processed_audio"):
    for file in os.listdir("processed_audio"):
        os.remove(os.path.join("processed_audio", file))
    os.rmdir("processed_audio")
    cleaned_files += len(processed_files)

print(f" Очищено файлов: {cleaned_files}")

print("\n Обработка завершена успешно!")
print(" Архив с обработанными файлами был скачан автоматически")